# Prototype and Initialisation File

In [ ]:
import math
import copy
import random
import allocationStructures
import numpy as np
import pandas as pd

# random seed

data = pd.read_csv('input_data/HSS Course Sem 2, 2023-2024 (Responses) - Form Responses 1.csv')

## Data Cleaning

In [ ]:
# display(data[[str(i)[-2:]!=str(j)[:2] for i, j in zip(data['Year of joining the current programme'].values, data['Roll No'].values)]])

# Removing first-year undergraduate students
ug = data[[(i or j or k or l) for i, j, k, l in zip(data['Programme'] == 'BTech', data['Programme'] == 'BTech- MTech Dual Degree', data['Programme'] == 'Dual Major BTech', data['Programme'] == 'BSc in Engineering')]]
ug = ug[ug['Year of joining the current programme'] == 2023]
data = data.drop(ug.index, axis=0)
# Removing illogical entries
mismatch_roll = data[data['Roll No'].astype(str) != data['Re-enter Roll No'].astype(str)]
data = data.drop(mismatch_roll.index, axis=0)

print('Removed entries:')
display(pd.concat([ug, mismatch_roll]))

print('Final Data:')
display(data)

## Courses' File Creation

In [ ]:
# Courses File Creation
courses = [y[82:82+y[82:].find(']')] for y in data.columns[[x.startswith('Indicate the priority for your choice of courses') for x in data.columns]]]
course_codes = [' '.join(x.split()[:2]) for x in courses]
course_names = [' '.join(x.split()[2:]) for x in courses]
print(course_codes)
print(course_names)
courses_df = pd.DataFrame(np.array([course_codes, course_names]).T, columns=['Code', 'Name']).set_index('Code').drop_duplicates()
courses_df['Capacity'] = 40
courses_df.loc[[x for x in courses_df.index if x.startswith('MS')], 'Capacity'] = 50
courses_df['Max Suggestive Capacity'] = (5 * courses_df['Capacity'])//4
courses_df.to_csv('input_data/courses.csv')
courses = [allocationStructures.Course(i, courses_df.loc[i]['Name'], courses_df.loc[i]['Capacity']) for i in courses_df.index]
course_dict = {}
for i, course in enumerate(courses):
    course_dict[course.code] = i
print(courses_df)
print(course_dict)

## Students' File Creation

In [ ]:
students = [allocationStructures.Student(str(stud[4]), ' '.join([('' if pd.isna(stud[2]) else stud[2].strip().upper()), ('' if pd.isna(stud[3]) else stud[3].strip().upper())])) for stud in data.values]
student_dict = {}
for i, stud in enumerate(students):
    student_dict[stud.roll] = i
pd.DataFrame([[stud.roll, stud.name] for stud in students]).to_csv('input_data/students.csv')

## MA Students' Core Course Allocation

In [ ]:
core_courses = [' '.join(x.split()[:2]) for x in [y[87:87+y[87:].find(']')] for y in data.columns[[x.startswith('Indicate the priority for your choice of core courses') for x in data.columns]]]]
ma_df = data[data['Programme'] == 'MA'][['Email Address', 'First Name', 'Last Name', 'Roll No', 'How Many core courses would you be taking this semester?'] + data.columns[[x.startswith('Indicate the priority for your choice of core courses') for x in data.columns]].tolist()]
for entry in ma_df.values:
    sroll = str(entry[3])
    dn = entry[4]
    pref = [-1 for i in range(len(core_courses))]
    for i, cc in enumerate(core_courses):
        if pd.notna(entry[i+5]):
            pref[int(entry[i+5].split()[-1]) - 1] = cc
    for i, cc in enumerate(pref):
        if cc != -1 and dn>0:
            students[student_dict[sroll]].allocate(courses[course_dict[cc]])
            dn -= 1
    print(students[student_dict[sroll]])

In [ ]:
for course in core_courses:
    print(courses[course_dict[course]])

## Management Minors Students' Allocation

In [ ]:
# Selecting students eligible for management minors preference
ug_stud = data[[i or j or k for i, j, k in zip(data['Programme'] == 'BTech', data['Programme'] == 'BTech- MTech Dual Degree', data['Programme'] == 'Dual Major BTech')]]
ug_stud = ug_stud[ug_stud['Are you pursuing a Minor in Management?'] == 'Yes']
ug_stud20 = ug_stud[ug_stud['Year of joining the current programme'] < 2021]
ug_stud21 = ug_stud[ug_stud['Year of joining the current programme'] == 2021]
ug_stud22 = ug_stud[ug_stud['Year of joining the current programme'] > 2021]
ug_stud = pd.concat([ug_stud20[ug_stud20['How many courses have you completed towards your Management Minor'] > 2], ug_stud21[ug_stud21['How many courses have you completed towards your Management Minor'] > 0], ug_stud22])

In [ ]:
pc = [(i, column) for i, column in enumerate(ug_stud.columns) if column.startswith('Indicate the priority for your choice of courses')]
courses_in_consideration = [course_dict[course.code] for course in courses if course.code.startswith('MS')]
course_columns = [[] for _ in range(len(courses_in_consideration))]
for ind, course in enumerate(courses_in_consideration):
    course_columns[ind] = [i[0] for i in pc if courses[course].code+' ' in ' '.join(i[1].split())]
students_in_consideration = [student_dict[str(entry[4])] for entry in ug_stud.values]
rem = []
for ind, entry in enumerate(ug_stud.values):
    pref = ['*' for _ in range(6)]
    for course, column_names in zip(courses_in_consideration, course_columns):
        for cn in column_names:
            if pd.notna(entry[cn]):
                pref[int(entry[cn].split()[-1]) - 1] = courses[course].code
    pref = [_ for _ in pref if _ != '*']
    if len(pref)==0:
        students_in_consideration.remove(student_dict[str(entry[4])])
        rem.append(ug_stud.index[ind])
    else:
        students[student_dict[str(entry[4])]].course_preferences = copy.copy(pref)
        students[student_dict[str(entry[4])]].desired_n = int(entry[12])
ug_stud.drop(rem, axis = 0, inplace=True)
display(ug_stud)

unassigned_students = [_ for _ in students_in_consideration]

f = 0
while len(unassigned_students) > 0:
    total = 0
    epoch = 0
    print(f'#{f+1} Assignment:')
    while len(unassigned_students) > 0 and epoch < 1000:
        lucky_student = random.choice(unassigned_students)
        for pref in students[lucky_student].course_preferences:
            if courses[course_dict[pref]].r_cap > 0:
                courses[course_dict[pref]].add_student(students[lucky_student])
                unassigned_students.remove(lucky_student)
                total += students[lucky_student].temp_score
                break
            elif len(courses[course_dict[pref]].tempAllocatedStudents) > 0 and len(courses[course_dict[pref]].tempAllocatedStudents[-1].course_preferences) > len(students[lucky_student].course_preferences):
                total -= students[student_dict[courses[course_dict[pref]].tempAllocatedStudents[-1].roll]].temp_score
                courses[course_dict[pref]].popularityIndex += students[student_dict[courses[course_dict[pref]].tempAllocatedStudents[-1].roll]].temp_score / students[student_dict[courses[course_dict[pref]].tempAllocatedStudents[-1].roll]].desired_n
                unlucky_student = courses[course_dict[pref]].remove_last()
                unassigned_students.append(student_dict[unlucky_student])
                courses[course_dict[pref]].add_student(students[lucky_student])
                unassigned_students.remove(lucky_student)
                total += students[lucky_student].temp_score
                break
            elif len(courses[course_dict[pref]].tempAllocatedStudents) > 0 and len(courses[course_dict[pref]].tempAllocatedStudents[-1].course_preferences) == len(students[lucky_student].course_preferences) and random.choice([0, 1]) == 1:
                total -= students[student_dict[courses[course_dict[pref]].tempAllocatedStudents[-1].roll]].temp_score
                courses[course_dict[pref]].popularityIndex += students[student_dict[courses[course_dict[pref]].tempAllocatedStudents[-1].roll]].temp_score / students[student_dict[courses[course_dict[pref]].tempAllocatedStudents[-1].roll]].desired_n
                unlucky_student = courses[course_dict[pref]].remove_last()
                unassigned_students.append(student_dict[unlucky_student])
                courses[course_dict[pref]].add_student(students[lucky_student])
                unassigned_students.remove(lucky_student)
                total += students[lucky_student].temp_score
                break
        epoch += 1
        print(f'#{epoch} epoch: Satisfaction score = {total}')
    for course in courses_in_consideration:
        courses[course].fixate()
    unassigned_students = [i for i in students_in_consideration if students[i].desired_n > students[i].actual_n and len(students[i].course_preferences) > 0]
    f += 1

In [ ]:
for stud in students_in_consideration:
    print(students[stud])

In [ ]:
for c in courses_in_consideration:
    print(courses[c])

## HSS Students' Preferred Allotment

In [ ]:
# Selecting students eligible for HSS preference
hs_stud = data[data['Discipline'] == 'Humanities and Social Sciences']
hs_stud = hs_stud[hs_stud['Programme'] != 'PhD']
display(hs_stud)

In [ ]:
pc = [(i, column) for i, column in enumerate(hs_stud.columns) if column.startswith('Indicate the priority for your choice of courses')]
courses_in_consideration = [course_dict[course.code] for course in courses if course.code.startswith('HS')]
course_columns = [[] for _ in range(len(courses_in_consideration))]
for ind, course in enumerate(courses_in_consideration):
    course_columns[ind] = [i[0] for i in pc if courses[course].code+' ' in ' '.join(i[1].split())]
students_in_consideration = [student_dict[str(entry[4])] for entry in hs_stud.values]
rem = []
for ind, entry in enumerate(hs_stud.values):
    pref = ['*' for _ in range(6)]
    for course, column_names in zip(courses_in_consideration, course_columns):
        for cn in column_names:
            if pd.notna(entry[cn]):
                pref[int(entry[cn].split()[-1]) - 1] = courses[course].code
    pref = [_ for _ in pref if _ != '*']
    pref = [_ for _ in pref if courses[course_dict[_]] not in students[student_dict[str(entry[4])]].allocated_courses]
    if len(pref)==0:
        students_in_consideration.remove(student_dict[str(entry[4])])
        rem.append(hs_stud.index[ind])
    else:
        students[student_dict[str(entry[4])]].course_preferences = copy.copy(pref)
        students[student_dict[str(entry[4])]].desired_n = max(0 if pd.isna(entry[12]) else int(entry[12]), 0 if pd.isna(entry[59]) else int(entry[59]))
hs_stud.drop(rem, axis = 0, inplace=True)
display(hs_stud)

unassigned_students = [_ for _ in students_in_consideration if students[_].desired_n > students[_].actual_n == 0]
print(len(unassigned_students))

f = 0
while f<2:
    total = 0
    epoch = 0
    print(f'#{f+1} Assignment:')
    while len(unassigned_students) > 0 and epoch < 1000:
        lucky_student = random.choice(unassigned_students)
        for pref in students[lucky_student].course_preferences:
            if courses[course_dict[pref]].r_cap > 0:
                courses[course_dict[pref]].add_student(students[lucky_student])
                unassigned_students.remove(lucky_student)
                total += students[lucky_student].temp_score
                break
            elif len(courses[course_dict[pref]].tempAllocatedStudents) > 0 and len(courses[course_dict[pref]].tempAllocatedStudents[-1].course_preferences) > len(students[lucky_student].course_preferences):
                total -= students[student_dict[courses[course_dict[pref]].tempAllocatedStudents[-1].roll]].temp_score
                courses[course_dict[pref]].popularityIndex += students[student_dict[courses[course_dict[pref]].tempAllocatedStudents[-1].roll]].temp_score / students[student_dict[courses[course_dict[pref]].tempAllocatedStudents[-1].roll]].desired_n
                unlucky_student = courses[course_dict[pref]].remove_last()
                unassigned_students.append(student_dict[unlucky_student])
                courses[course_dict[pref]].add_student(students[lucky_student])
                unassigned_students.remove(lucky_student)
                total += students[lucky_student].temp_score
                break
            elif len(courses[course_dict[pref]].tempAllocatedStudents) > 0 and len(courses[course_dict[pref]].tempAllocatedStudents[-1].course_preferences) == len(students[lucky_student].course_preferences) and random.choice([0, 1]) == 1:
                total -= students[student_dict[courses[course_dict[pref]].tempAllocatedStudents[-1].roll]].temp_score
                courses[course_dict[pref]].popularityIndex += students[student_dict[courses[course_dict[pref]].tempAllocatedStudents[-1].roll]].temp_score / students[student_dict[courses[course_dict[pref]].tempAllocatedStudents[-1].roll]].desired_n
                unlucky_student = courses[course_dict[pref]].remove_last()
                unassigned_students.append(student_dict[unlucky_student])
                courses[course_dict[pref]].add_student(students[lucky_student])
                unassigned_students.remove(lucky_student)
                total += students[lucky_student].temp_score
                break
        epoch += 1
        print(f'#{epoch} epoch: Satisfaction score = {total}')
    for course in courses_in_consideration:
        courses[course].fixate()
    unassigned_students = [i for i in students_in_consideration if (students[i].desired_n > students[i].actual_n <= f+1) and len(students[i].course_preferences) > 0]
    f += 1
    print()

In [ ]:
for stud in students_in_consideration:
    print(students[stud])

In [ ]:
for c in courses_in_consideration:
    print(courses[c])

## General Allocation (excluding PhD HSS)

In [ ]:
# Excluding PhD students
phd_stud = data[data['Programme'] == 'PhD']
phd_stud = phd_stud[phd_stud['Discipline'] == 'Humanities and Social Sciences']
g_stud = data.drop(phd_stud.index, axis = 0)
display(g_stud)

In [ ]:
pc = [(i, column) for i, column in enumerate(g_stud.columns) if column.startswith('Indicate the priority for your choice of courses')]
courses_in_consideration = [course_dict[course.code] for course in courses]
course_columns = [[] for _ in range(len(courses_in_consideration))]
for ind, course in enumerate(courses_in_consideration):
    course_columns[ind] = [i[0] for i in pc if courses[course].code+' ' in ' '.join(i[1].split())]
students_in_consideration = [student_dict[str(entry[4])] for entry in g_stud.values]
rem = []
for ind, entry in enumerate(g_stud.values):
    pref = ['*' for _ in range(6)]
    for course, column_names in zip(courses_in_consideration, course_columns):
        for cn in column_names:
            if pd.notna(entry[cn]):
                pref[int(entry[cn].split()[-1]) - 1] = courses[course].code
    pref = [_ for _ in pref if _ != '*']
    pref = [_ for _ in pref if courses[course_dict[_]] not in students[student_dict[str(entry[4])]].allocated_courses]
    if len(pref)==0:
        students_in_consideration.remove(student_dict[str(entry[4])])
        rem.append(g_stud.index[ind])
    else:
        students[student_dict[str(entry[4])]].course_preferences = copy.copy(pref)
        students[student_dict[str(entry[4])]].desired_n = max(0 if pd.isna(entry[12]) else int(entry[12]), 0 if pd.isna(entry[59]) else int(entry[59]))
g_stud.drop(rem, axis = 0, inplace=True)
display(g_stud)

unassigned_students = [_ for _ in students_in_consideration if students[_].desired_n > students[_].actual_n == 0]
print(len(unassigned_students))

f = 0
while f<7:
    total = 0
    epoch = 0
    print(f'#{f+1} Assignment:')
    while len(unassigned_students) > 0 and epoch < 5000:
        lucky_student = random.choice(unassigned_students)
        for pref in students[lucky_student].course_preferences:
            if courses[course_dict[pref]].r_cap > 0:
                courses[course_dict[pref]].add_student(students[lucky_student])
                unassigned_students.remove(lucky_student)
                total += students[lucky_student].temp_score
                break
            elif len(courses[course_dict[pref]].tempAllocatedStudents) > 0 and len(courses[course_dict[pref]].tempAllocatedStudents[-1].course_preferences) > len(students[lucky_student].course_preferences):
                total -= students[student_dict[courses[course_dict[pref]].tempAllocatedStudents[-1].roll]].temp_score
                courses[course_dict[pref]].popularityIndex += students[student_dict[courses[course_dict[pref]].tempAllocatedStudents[-1].roll]].temp_score / students[student_dict[courses[course_dict[pref]].tempAllocatedStudents[-1].roll]].desired_n
                unlucky_student = courses[course_dict[pref]].remove_last()
                unassigned_students.append(student_dict[unlucky_student])
                courses[course_dict[pref]].add_student(students[lucky_student])
                unassigned_students.remove(lucky_student)
                total += students[lucky_student].temp_score
                break
            elif len(courses[course_dict[pref]].tempAllocatedStudents) > 0 and len(courses[course_dict[pref]].tempAllocatedStudents[-1].course_preferences) == len(students[lucky_student].course_preferences) and random.choice([0, 1]) == 1:
                total -= students[student_dict[courses[course_dict[pref]].tempAllocatedStudents[-1].roll]].temp_score
                courses[course_dict[pref]].popularityIndex += students[student_dict[courses[course_dict[pref]].tempAllocatedStudents[-1].roll]].temp_score / students[student_dict[courses[course_dict[pref]].tempAllocatedStudents[-1].roll]].desired_n
                unlucky_student = courses[course_dict[pref]].remove_last()
                unassigned_students.append(student_dict[unlucky_student])
                courses[course_dict[pref]].add_student(students[lucky_student])
                unassigned_students.remove(lucky_student)
                total += students[lucky_student].temp_score
                break
        epoch += 1
        if epoch%50 == 0:
            print(f'#{epoch} epoch: Satisfaction score = {total}')
    for course in courses_in_consideration:
        courses[course].fixate()
    unassigned_students = [i for i in students_in_consideration if (students[i].desired_n > students[i].actual_n <= f+1) and len(students[i].course_preferences) > 0]
    f += 1
    print()

In [ ]:
for stud in students_in_consideration:
    print(students[stud])

In [ ]:
for c in courses_in_consideration:
    print(courses[c])

## PhD Students in HSS allocation

In [ ]:
pc = [(i, column) for i, column in enumerate(phd_stud.columns) if column.startswith('Indicate the priority for your choice of courses')]
courses_in_consideration = [course_dict[course.code] for course in courses]
course_columns = [[] for _ in range(len(courses_in_consideration))]
for ind, course in enumerate(courses_in_consideration):
    course_columns[ind] = [i[0] for i in pc if courses[course].code+' ' in ' '.join(i[1].split())]
students_in_consideration = [student_dict[str(entry[4])] for entry in phd_stud.values]
rem = []
for ind, entry in enumerate(phd_stud.values):
    pref = ['*' for _ in range(6)]
    for course, column_names in zip(courses_in_consideration, course_columns):
        for cn in column_names:
            if pd.notna(entry[cn]):
                pref[int(entry[cn].split()[-1]) - 1] = courses[course].code
    pref = [_ for _ in pref if _ != '*']
    pref = [_ for _ in pref if courses[course_dict[_]] not in students[student_dict[str(entry[4])]].allocated_courses]
    if len(pref)==0:
        students_in_consideration.remove(student_dict[str(entry[4])])
        rem.append(phd_stud.index[ind])
    else:
        students[student_dict[str(entry[4])]].course_preferences = copy.copy(pref)
        students[student_dict[str(entry[4])]].desired_n = max(0 if pd.isna(entry[12]) else int(entry[12]), 0 if pd.isna(entry[59]) else int(entry[59]))
phd_stud.drop(rem, axis = 0, inplace=True)
display(phd_stud)

for stud in students_in_consideration:
    while len(students[stud].course_preferences) > 0 and students[stud].actual_n<students[stud].desired_n:
        courses[course_dict[students[stud].course_preferences[0]]].add_super_student(students[stud])

In [ ]:
for stud in students_in_consideration:
    print(students[stud])

In [ ]:
for c in courses_in_consideration:
    print(courses[c])

## Saving as files

In [ ]:
output = pd.DataFrame(data[['Email Address', 'First Name', 'Last Name', 'Roll No', 'Alternate Email id', 'Discipline', 'Year of joining the current programme', 'Programme']])
al_courses = []
for roll in output['Roll No']:
    al_courses.append(list(map(lambda x: x.code, students[student_dict[str(roll)]].allocated_courses)))
output['Allocated Courses'] = pd.Series(al_courses).values
output.to_csv('output_data/StudentWiseAllocation.csv')

In [ ]:
coutput = pd.read_csv('input_data/courses.csv')[['Code', 'Name']]
al_stud = []
al_n = []
al_sn = []
for code in coutput['Code']:
    al_stud.append(list(map(lambda x: x.roll, courses[course_dict[code]].permAllocatedStudents)))
    al_n.append(courses[course_dict[code]].cap-courses[course_dict[code]].r_cap+courses[course_dict[code]].sn)
    al_sn.append(courses[course_dict[code]].sn)
coutput['Number of allocated Students'] = pd.Series(al_n).values
coutput['Number of supernumerary Students'] = pd.Series(al_sn).values
coutput['Allocated Students'] = pd.Series(al_stud).values
coutput.to_csv('output_data/CourseWiseAllocation.csv')

## Student Satisfaction Scores

In [ ]:
mini = maxi = students[0].score
total = 0
for stud in students:
    print(f'{stud.roll}\t{stud.name}\t{stud.score}')
    mini = min(mini, stud.score)
    maxi = max(maxi, stud.score)
    total += stud.score
print()
print(f'{total}\n{mini}\n{maxi}')

## Courses Popularity Index

In [ ]:
sorted_courses = courses.copy()
sorted_courses.sort(key=lambda x:x.popularityIndex, reverse=True)
print(*sorted_courses, sep='\n\n')

In [ ]:
students[student_dict['19110206']]